<a href="https://colab.research.google.com/github/Az-Ks/AirQo-Ugandan-Air-Quality-Forecast/blob/master/EKHERMOU7AWLA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install catboost

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from math import sqrt 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold, train_test_split


from scipy.sparse import csr_matrix
import gc

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,BaggingRegressor,AdaBoostRegressor,ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
#from ngboost import NGBRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold,StratifiedKFold, GroupKFold
import datetime
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# from fastai.tabular import *
import warnings
warnings.filterwarnings("ignore")
np.random.seed(0)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# Memory reduction helper function:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #columns
        col_type = df[col].dtypes
        if col_type in numerics: #numerics
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [0]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "rf.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
path =  '/content/drive/My Drive/'

# THIS IS THE NEW DATA  GENERATED IN NEW_DATA NOTEBOOK 

In [0]:
train = pd.read_csv(path+'train_df.csv')
test = pd.read_csv(path+'test_df.csv')
train.shape,test.shape

((15539, 3087), (5035, 3086))

In [0]:
# train = reduce_mem_usage(train)
# test = reduce_mem_usage(test)

In [0]:
temp = ["temp{}".format(i) for i in range(121)]
precip = ["precip{}".format(i) for i in range(121)]
rel_humidity = ["rel_humidity{}".format(i) for i in range(121)]
wind_dir = ["wind_dir{}".format(i) for i in range(121)]
wind_spd = ["wind_spd{}".format(i) for i in range(121)]
atmos_press = ["atmos_press{}".format(i) for i in range(121)]

In [0]:
grp_data11_col = pd.read_csv(path+'grp_data11_col.csv')
grp_data11_col = list(grp_data11_col['0'].values)

grp_data21_col = pd.read_csv(path+'grp_data21_col.csv')
grp_data21_col = list(grp_data21_col['0'].values)

grp_data31_col = pd.read_csv(path+'grp_data31_col.csv')
grp_data31_col = list(grp_data31_col['0'].values)

grp_data41_col = pd.read_csv(path+'grp_data41_col.csv')
grp_data41_col = list(grp_data41_col['0'].values)

grp_data51_col = pd.read_csv(path+'grp_data51_col.csv')
grp_data51_col = list(grp_data51_col['0'].values)

rem_org = pd.read_csv(path+'rem_org.csv')
rem_org = list(rem_org['0'].values)

In [0]:
len(rem_org),len(grp_data11_col),len(grp_data21_col),len(grp_data31_col),len(grp_data41_col),len(grp_data51_col)

(116, 511, 409, 817, 307, 205)

In [0]:
f1 = list(set(temp + precip + rel_humidity + wind_dir + wind_spd + atmos_press + rem_org)) # orig
f2 = list(set(f1 + grp_data11_col)) #day
f3 = list(set(f1 + grp_data21_col)) #typeofday
f4 = list(set(f1 + grp_data31_col)) #type_of_day1
f5 = list(set(f1 + grp_data41_col)) #type_of_day2
f6 = list(set(f1 + grp_data51_col)) #type_of_day3
f7 = list(set(f1 + f3 + f5)) #type_of_day day_type_of_day2
f8 = list(set(f1 + f3 + f6)) #type_of_day day_type_of_day3
f9 = list(set(f1 + f4 + f5)) #type_of_day1 day_type_of_day2
f10 = list(set(f1 + f4 + f6)) #type_of_day1 day_type_of_day3
f11 = list(set(f1 + f5 + f6)) #type_of_day2 day_type_of_day3
f12 = list(set(f1 + f3+ f5 + f6)) #type_of_day type_of_day2 day_type_of_day3
f13 = list(set(f1 + f4+ f5 + f6)) #type_of_day1 type_of_day2 day_type_of_day3
f14 = list(set(f1 + f4+ f5 + f3 + f6)) #type_of_day1 type_of_day2 day_type_of_day3

In [0]:
train = train[f14+['target']]
test = test[f14]
train.shape,test.shape

((15539, 2577), (5035, 2576))

In [0]:
train_id = train['ID']
test_id = test['ID']
y = train['target']

train.drop(['ID','target'],axis=1,inplace=True)
test.drop('ID',axis=1,inplace=True)
train.shape,test.shape

((15539, 2575), (5035, 2575))

## THIS IS THE SAME DATA GENERATED BY ARIMA 
# **PLEASE CHECK THAT YOU'VE UPLOAD IT **

In [0]:
forecastwindte=pd.read_csv('/content/winddirforecasttest ADD THOSE FEATURES TO TEST.csv')
forecastwindtr=pd.read_csv('/content/winddirforecasttrain ADD THOSE COLUMNS TO TRAIN.csv')

In [0]:
X  = train.copy()
Xtest = test.copy()

In [0]:
for i in forecastwindte.columns :
  X[i]=forecastwindtr[i]
  Xtest[i]=forecastwindte[i]

In [0]:
%%time
from lightgbm import LGBMRegressor
lgb = LGBMRegressor()
lgb.fit(X,y)

imp = pd.DataFrame(lgb.feature_importances_,index=X.columns)
l = list(imp[imp[0]>3].index)
len(l)

CPU times: user 1min 51s, sys: 571 ms, total: 1min 51s
Wall time: 57.5 s


In [0]:
len(l),X.shape

(205, (15539, 2593))

In [0]:
X["wind_dir_percentile_10Evening"]

In [0]:
X['relation1'] = X['wind_dir_percentile_10Noon']  / X["wind_dir_percentile_10Mor2"]
X['relation2'] = X['rel_humidity_percentile_40Noon'] / X['rel_humidity_percentile_40Mor2']
X['relation3'] = X['wind_dir_percentile_70Noon']  / X['wind_dir_percentile_70Mor2']
X['relation4'] =X['atmos_press_percentile_90Noon']  / X['atmos_press_percentile_90Mor2']
X['relation5'] = X['rel_humidity_maxNoon'] / X['rel_humidity_maxMor2']
X['relation6']  = X['temp_stdNoon']  /X['temp_stdMor2']
X['relation7']  = X['rel_humidity_minNight'] / X['rel_humidity_minEvening']
X['relation8']  = X["wind_dir_maxNight"] / X['wind_dir_maxEvening']

In [0]:
Xtest['relation1'] = Xtest['wind_dir_percentile_10Noon']  / Xtest["wind_dir_percentile_10Mor2"]
Xtest['relation2'] = Xtest['rel_humidity_percentile_40Noon'] / Xtest['rel_humidity_percentile_40Mor2']
Xtest['relation3'] = Xtest['wind_dir_percentile_70Noon']  / Xtest['wind_dir_percentile_70Mor2']
Xtest['relation4'] =Xtest['atmos_press_percentile_90Noon']  / Xtest['atmos_press_percentile_90Mor2']
Xtest['relation5'] = Xtest['rel_humidity_maxNoon'] / Xtest['rel_humidity_maxMor2']
Xtest['relation6']  = Xtest['temp_stdNoon']  /Xtest['temp_stdMor2']
Xtest['relation7']  = Xtest['rel_humidity_minNight'] / Xtest['rel_humidity_minEvening']
Xtest['relation8']  = Xtest["wind_dir_maxNight"] / Xtest['wind_dir_maxEvening']

In [0]:
l+=['relation1','relation2','relation3','relation4','relation5','relation6','relation7','relation8',]

In [0]:
Xtest[l].describe()

,wind_dir_percentile_10Noon,wind_dir_stdEve1,wind_spd63,p30_wind_spd,wind_spd_percentile_70Nig2,p1_temp,max_wind_dir,p60_wind_dir,wind_dir_stdEvening,atmos_press113,temp_percentile_99Second,wind_dir_percentile_40First,atmos_press_percentile_95Nig1,min_rel_humidity,temp_percentile_95Third,wind_dir120,temp47,wind_dir3,wind_dir2,ptp_wind_spd,atmos_press_percentile_90Noo1,p20_wind_dir,p40_temp,p20_temp,p80_rel_humidity,temp_stdMor2,wind_spd_percentile_99S_half,wind_dir54,p1_wind_dir,temp_maxNoo1,temp_maxS_half,var_wind_dir,median_wind_spd,precip_percentile_99S_half,wind_dir_percentile_70Second,wind_dir_maxEvening,p60_rel_humidity,rel_humidity74,wind_dir_percentile_90Eve2,wind_spd38,wind_spd_percentile_5Evening,rel_humidity1,rel_humidity85,precip10,atmos_press_stdSecond,rel_humidity_percentile_90Mor2,rel_humidity_minNight,p5_wind_dir,wind_spd41,std_atmos_press,rel_humidity86,wind_dir_percentile_1Noo2,rel_humidity22,rel_humidity_percentile_40Noon,min_wind_spd,rel_humidity_percentile_99Noo1,precip_meanMorning,std_precip,p40_wind_dir,ptp_atmos_press,wind_dir7,p1_rel_humidity,temp95,atmos_press_stdMor1,wind_dir_percentile_70Noon,atmos_press_percentile_95Noo1,temp_percentile_60Noo1,temp_maxSecond,wind_spd120,std_rel_humidity,p80_atmos_press,temp10,mean_precip,wind_dir_percentile_80Morning,temp_maxF_half,std_temp,wind_dir_percentile_90S_half,p80_wind_dir,atmos_press62,wind_spd10,wind_dir_percentile_40F_half,p1_wind_spd,wind_dir_stdEve2,mean_wind_dir,rel_humidity_maxNoon,atmos_press86,atmos_press_percentile_90Night,p30_rel_humidity,wind_dir_percentile_99Nig1,wind_dir32,p70_wind_dir,temp52,p60_temp,precip_percentile_95S_half,p99_precip,wind_dir_stdNoo1,wind_dir_percentile_20Noo1,wind_dir_percentile_1S_half,wind_spd_stdNoo2,wind_dir_percentile_95F_half,p95_wind_spd,wind_dir87,p20_rel_humidity,p90_wind_spd,wind_dir_percentile_60F_half,temp_stdNight,wind_dir26,min_temp,temp_stdNig2,precip_stdFirst,wind_dir27,wind_dir_maxMor1,temp1,wind_dir_percentile_99Eve2,wind_dir_percentile_10Third,p30_wind_dir,ptp_precip,p60_wind_spd,precip_percentile_95Morning,wind_dir59,wind_dir105,wind_dir_stdNoo2,temp_percentile_95F_half,atmos_press_percentile_90Noon,temp_maxFirst,var_temp,p10_wind_spd,temp49,var_atmos_press,p80_wind_spd,rel_humidity_percentile_80F_half,wind_dir_percentile_80Evening,rel_humidity_stdF_half,wind_dir_stdSecond,wind_dir_percentile_10Noo2,wind_spd16,p95_wind_dir,temp115,mean_rel_humidity,wind_dir_percentile_90F_half,temp_percentile_99F_half,wind_dir34,temp_maxNig2,wind_dir_percentile_20Mor2,p99_rel_humidity,p10_wind_dir,atmos_press_stdNig1,rel_humidity107,p95_precip,wind_spd_stdF_half,p5_temp,p20_wind_spd,p99_wind_spd,p40_wind_spd,rel_humidity_minMor1,max_rel_humidity,p40_rel_humidity,wind_dir_maxMorning,wind_dir_percentile_10S_half,temp70,rel_humidity56,p70_rel_humidity,p99_wind_dir,p5_wind_spd,rel_humidity95,p99_temp,p95_rel_humidity,precip_stdNight,temp_stdMor1,wind_spd_maxSecond,wind_spd_meanS_half,temp_percentile_99Night,temp_percentile_20S_half,median_wind_dir,precip6,rel_humidity_stdMor1,median_temp,temp120,rel_humidity78,rel_humidity_percentile_99Second,rel_humidity_percentile_95F_half,rel_humidity120,atmos_press_stdNoon,wind_spd23,rel_humidity73,wind_spd_meanNig2,wind_spd_percentile_60Nig2,min_wind_dir,wind_spd_percentile_80Mor2,p90_wind_dir,ptp_temp,p70_temp,ptp_rel_humidity,atmos_press_stdNight,median_rel_humidity,temp_percentile_99Mor1,wind_spd_stdFirst,wind_spd_percentile_95S_half,wind_dir_meanF_half,wind_dir10,wind_dir30,wind_dir118,p90_temp,0,1,relation1,relation2,relation3,relation4,relation5,relation6,relation7,relation8
count,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.00

In [0]:
Xtest=Xtest[l]

In [0]:
X=X[l]

In [0]:
X[l].shape

(15539, 213)

In [0]:
from catboost import CatBoostRegressor
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
fold=KFold(n_splits=19)#15#5#10
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X[l].values[train_index], X[l].values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]
    m2  = CatBoostRegressor(n_estimators=5000,eval_metric='RMSE',learning_rate=0.175, random_seed= 42, use_best_model=True )
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=200,verbose=200)
    preds=m2.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    errcb2.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = m2.predict(Xtest[l])
    y_pred_totcb2.append(p2)
np.mean(errcb2)

0:	learn: 40.1815492	test: 40.1815492	test1: 40.9694681	best: 40.9694681 (0)	total: 71ms	remaining: 5m 55s
200:	learn: 18.5038530	test: 18.5038530	test1: 24.6823494	best: 24.6823494 (200)	total: 13.4s	remaining: 5m 19s
400:	learn: 13.9539272	test: 13.9539272	test1: 23.2929758	best: 23.2896084 (395)	total: 26.7s	remaining: 5m 5s
600:	learn: 11.2003063	test: 11.2003063	test1: 22.8471532	best: 22.8471532 (600)	total: 40s	remaining: 4m 52s
800:	learn: 9.1825672	test: 9.1825672	test1: 22.5385248	best: 22.5360729 (799)	total: 53.3s	remaining: 4m 39s
1000:	learn: 7.6023115	test: 7.6023115	test1: 22.3484841	best: 22.3457678 (995)	total: 1m 6s	remaining: 4m 25s
1200:	learn: 6.3879740	test: 6.3879740	test1: 22.3094131	best: 22.3033702 (1167)	total: 1m 19s	remaining: 4m 12s
1400:	learn: 5.4401416	test: 5.4401416	test1: 22.2713607	best: 22.2586499 (1344)	total: 1m 33s	remaining: 3m 59s
1600:	learn: 4.6254015	test: 4.6254015	test1: 22.1953915	best: 22.1953915 (1600)	total: 1m 46s	remaining: 3m 46s


KeyboardInterrupt: ignored

In [0]:
np.mean(errcb2)

22.68873411332329

In [0]:
d = {'ID': test_id, 'target': np.mean(y_pred_totcb2, 0)}
sub = pd.DataFrame(data=d)
sub = sub[['ID', 'target']]

In [0]:
sub.to_csv('ekher_mou7awla.csv',index=False)
from google.colab import files
files.download("ekher_mou7awla.csv")